In [1]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="1mo")
print(hist)

f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


                                 Open        High         Low       Close  \
Date                                                                        
2024-02-09 00:00:00-05:00  414.483507  420.043233  414.323799  419.773712   
2024-02-12 00:00:00-05:00  419.783705  419.963365  413.984431  414.493500   
2024-02-13 00:00:00-05:00  404.192550  409.313085  402.645423  405.570007   
2024-02-14 00:00:00-05:00  408.070007  409.839996  404.570007  409.489990   
2024-02-15 00:00:00-05:00  408.140015  409.130005  404.290009  406.559998   
2024-02-16 00:00:00-05:00  407.959991  408.290009  403.440002  404.059998   
2024-02-20 00:00:00-05:00  403.239990  404.489990  398.010010  402.790009   
2024-02-21 00:00:00-05:00  400.170013  402.290009  397.220001  402.179993   
2024-02-22 00:00:00-05:00  410.190002  412.829987  408.570007  411.649994   
2024-02-23 00:00:00-05:00  415.670013  415.859985  408.970001  410.339996   
2024-02-26 00:00:00-05:00  411.459991  412.160004  407.359985  407.540009   

In [2]:
import yfinance as yf


def get_stock_data(symbol):
    # Create a Ticker object for the given stock symbol
    stock = yf.Ticker(symbol)

    # Get historical market data for the last one month
    hist_data = stock.history(period="1mo")

    return hist_data


# Example usage
stock_symbol = "MSFT"
stock_data = get_stock_data(stock_symbol)
print(stock_data)

                                 Open        High         Low       Close  \
Date                                                                        
2024-02-09 00:00:00-05:00  414.483507  420.043233  414.323799  419.773712   
2024-02-12 00:00:00-05:00  419.783705  419.963365  413.984431  414.493500   
2024-02-13 00:00:00-05:00  404.192550  409.313085  402.645423  405.570007   
2024-02-14 00:00:00-05:00  408.070007  409.839996  404.570007  409.489990   
2024-02-15 00:00:00-05:00  408.140015  409.130005  404.290009  406.559998   
2024-02-16 00:00:00-05:00  407.959991  408.290009  403.440002  404.059998   
2024-02-20 00:00:00-05:00  403.239990  404.489990  398.010010  402.790009   
2024-02-21 00:00:00-05:00  400.170013  402.290009  397.220001  402.179993   
2024-02-22 00:00:00-05:00  410.190002  412.829987  408.570007  411.649994   
2024-02-23 00:00:00-05:00  415.670013  415.859985  408.970001  410.339996   
2024-02-26 00:00:00-05:00  411.459991  412.160004  407.359985  407.540009   

f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [3]:
import yfinance as yf
import psycopg2

hostname = "localhost"
database = "Finance"
username = "postgres"
pwd = "1234"
port_id = 5432

conn = None
curr = None

try:
    # Establish connection to PostgreSQL
    conn = psycopg2.connect(
        host=hostname, dbname=database, user=username, password=pwd, port=port_id
    )
    curr = conn.cursor()

    # Create the stock table if it doesn't exist
    create_script = """CREATE TABLE IF NOT EXISTS stock (
                    date_time TIMESTAMP PRIMARY KEY,
                    open_price FLOAT,
                    high_price FLOAT,
                    low_price FLOAT,
                    close_price FLOAT
                    )"""
    curr.execute(create_script)
    conn.commit()

    # Get historical market data for the last one month
    msft = yf.Ticker("MSFT")
    hist = msft.history(period="1mo")

    # Insert data into the stock table
    for index, row in hist.iterrows():
        curr.execute(
            "INSERT INTO stock (date_time, open_price, high_price, low_price, close_price) VALUES (%s, %s, %s, %s, %s)",
            (index, row["Open"], row["High"], row["Low"], row["Close"]),
        )
    conn.commit()

except Exception as error:
    print(error)

finally:
    if curr is not None:
        curr.close()
    if conn is not None:
        conn.close()

duplicate key value violates unique constraint "stock_pkey"
DETAIL:  Key (date_time)=(2024-02-09 10:30:00) already exists.



f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
